# ALL_RSS

## Given that RSS FEEDs update frequently we need to find a way to include the new jobs into the current DB without losing the "old" jobs.

### In brief...

### For every iteration, the df is inspected and then filtered by its pubDate (if there's one). Finally, it creates a new df that will be transformed into a new table in the local postgreSQL DB.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import mysql.connector
import psycopg2
import numpy as np

In [2]:
df = pd.read_csv('/Users/juanreyesgarcia/Library/CloudStorage/OneDrive-FundacionUniversidaddelasAmericasPuebla/DEVELOPER/PROJECTS/CRAWLER_ALL/OUTPUTS/yummy_soup_rss.csv')

##### Play with the settings...

In [3]:
pd.set_option('display.max_colwidth', 150)

In [4]:
pd.set_option("display.max_rows", None)

# EDA Checklist 

In [5]:
df.head()

,title,link,description,pubdate,location
0,Clojure Scala Java Engineer Deep Impact,https://ClojureJobboard.com/clojure-job/hz-clojure-scala-java-engineer-zurich-deep-impact.html,Zurich Switzerland We are looking for a talented Software Engineer with experience in Clojure or Scala or Java to join our product team to build ...,NaN,NaN
1,Backend Engineer Clojure Snap eHealth,https://ClojureJobboard.com/clojure-job/io-remote-backend-engineer-clojure-denver-snap-ehealth-remotework.html,Remote Snap eHealth is a wellfunded newly formed healthtech startup helping patients share their healthcare data with their physicians Our softwa...,NaN,NaN
2,Senior Clojure Developer Codete,https://ClojureJobboard.com/clojure-job/ib-remote-senior-clojure-developer-krakow-codete-remotework.html,Remote Codete is not just a software company its a place where techenthusiasts can grow by doing what they love and feel valued for what they are...,NaN,NaN
3,Senior Software Engineer CIS Cloud Services Centripetal,https://ClojureJobboard.com/clojure-job/ha-remote-senior-software-engineer-cis-cloud-services-centripetal-remotework.html,Remote Do you want to be at the forefront of intelligencedriven cybersecurity We at Centripetal are innovators of disruptive cybersecurity soluti...,NaN,NaN
4,FullStack Clojure Developer Appear,https://ClojureJobboard.com/clojure-job/ia-remote-europe-full-stack-clojure-developer-appear-remotework.html,Remote At Appear were looking for a FullStack Clojure Developer to join our team Fulltime Oslo OR Remote About the role Were looking for a Cloju...,NaN,NaN


In [6]:
df.tail()

,title,link,description,pubdate,location
1375,rtCamp WordPress Trainer,https://jobs.wordpress.net/job/rtcamp-wordpress-trainer-22/,Job description You will be working in our Learning and Development Team with a focus on training freshers hired through our campus program aiming...,Mon 06 Mar 2023 162923 0000,NaN
1376,WordPress tech lead in a Danish company The next career step,https://jobs.wordpress.net/job/wordpress-tech-lead-in-a-danish-company-the-next-career-step/,WordPress tech lead COMPANY Webnorth ApS httpswebnorthcom LOCATION Remote TEAM WP team EMPLOYMENT Fulltime EXPERIENCE Min 5 years of experience Ab...,Mon 06 Mar 2023 162914 0000,NaN
1377,rtCamp Senior WordPress Engineer,https://jobs.wordpress.net/job/rtcamp-senior-wordpress-engineer-53/,Note This is a fulltime W2 remote opportunity for US residents along with benefits and an annual pay ranging from 80k100k 4050hr Job description ...,Mon 06 Mar 2023 123325 0000,NaN
1378,WordPress Designer,https://jobs.wordpress.net/job/wordpress-designer-54/,ob Title WordPress Designer for httpjomirecordonline Job Description We are seeking a skilled and experienced WordPress Designer to join our team ...,Sun 05 Mar 2023 141018 0000,NaN
1379,WordPress Developer,https://jobs.wordpress.net/job/wordpress-developer-793/,Job Title WordPress Developer for httpjomerecordonline Job Description We are seeking a talented and experienced WordPress Developer to join our t...,Sun 05 Mar 2023 141009 0000,NaN


### Describe the DF (current)

In [7]:
df.describe()

,title,link,description,pubdate,location
count,1380,1380,1134,1201,636
unique,1253,1380,1101,1135,181
top,DevOps Engineer,https://ClojureJobboard.com/clojure-job/hz-clojure-scala-java-engineer-zurich-deep-impact.html,United States,Wed 08 Mar 2023 080000 0200,Remote
freq,7,1,11,7,80


### Checking for missing values DF (current)

In [8]:
df.isnull().sum()

title            0
link             0
description    246
pubdate        179
location       744
dtype: int64

### Fill missing values with "NaN" DF (current)

In [9]:
# Fill missing values with "NaN"
df.fillna("NaN", inplace=True)

In [10]:
df.isnull().sum()

title          0
link           0
description    0
pubdate        0
location       0
dtype: int64

### Checking for duplicates DF (current)

In [11]:
df.duplicated().sum()

0

In [12]:
df.describe()

,title,link,description,pubdate,location
count,1380,1380,1380,1380,1380
unique,1253,1380,1102,1136,182
top,DevOps Engineer,https://ClojureJobboard.com/clojure-job/hz-clojure-scala-java-engineer-zurich-deep-impact.html,NaN,NaN,NaN
freq,7,1,246,179,744


In [13]:
df.dtypes

title          object
link           object
description    object
pubdate        object
location       object
dtype: object

# From string to date time...

In [14]:
df.loc[:, 'pubdate']  = pd.to_datetime(df['pubdate'], errors="coerce", format="%a %d %b %Y", exact=False)

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_2525/3980262668.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'pubdate']  = pd.to_datetime(df['pubdate'], errors="coerce", format="%a %d %b %Y", exact=False)


## Filter rows by a date range (this reduces the number of rows... duh)

In [15]:
# filter rows by a date range
start_date = pd.to_datetime('2016-01-01')
end_date = pd.to_datetime('2023-02-15')

#for df
date_range_filter = (df['pubdate'] >= start_date) & (df['pubdate'] <= end_date)
df = df.loc[~date_range_filter]

In [16]:
df = df.sort_values(by='pubdate')

In [17]:
df.describe(datetime_is_numeric=True)

,pubdate
count,443
mean,2023-02-28 16:54:10.564334080
min,2023-02-16 00:00:00
25%,2023-02-23 00:00:00
50%,2023-03-02 00:00:00
75%,2023-03-06 00:00:00
max,2023-03-08 00:00:00


In [18]:
df

,title,link,description,pubdate,location
70,Principal software engineer SQL API at Cube Dev San Francisco CA USA,https://www.crunchboard.com/jobs/197021714-principal-software-engineer-sql-api-at-cube-dev,At Cube Dev we are building a technology stack for modern analytics If you are fascinated by the software that powers large technology companies but,2023-02-16,NaN
1342,Aha Senior Product Marketing Manager,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Headquarters United States URL httpwwwahaio Aha helps companies build lovable software We provide the worlds 1 product development tools Aha Ro...,2023-02-16,NaN
69,Senior Full Stack Developer NET at AAOS Rosemont IL USA,https://www.crunchboard.com/jobs/197057956-senior-full-stack-developer-net-at-aaos,You are an enthusiastic thought leader who coaches and mentors those around you You are highly creative and understand how to balance creativity and,2023-02-16,NaN
804,Voormedia is hiring for a position of Product Designer in Amsterdam Netherlands,https://dribbble.com/jobs/203058-Product-Designer?source=feed,NaN,2023-02-16,NaN
803,Johns Hopkins University is hiring for a position of Web Designer in Remote,https://dribbble.com/jobs/203090-Web-Designer?source=feed,NaN,2023-02-16,NaN
799,Riva Health is hiring for a position of Lead Product Designer Riva Health in United States Canada,https://dribbble.com/jobs/203123-Lead-Product-Designer-Riva-Health?source=feed,NaN,2023-02-16,NaN
883,Drupal FrontEnd Engineer Remote US amp CANADA position is open kanopistudios,https://jobs.drupal.org/kanopi-studios/job/17548,United States,2023-02-16,NaN
1257,Full Stack Developer,https://vuejobs.com/jobs/my-worksheet-maker-full-stack-developer?utm_source=rssfeed,Employer My Worksheet Maker Location Full Stack Developer FullTime RemotenbspAre you a skilled developer looking to use your technical expertise ...,2023-02-16,NaN
1256,UX and Design Focused Senior Frontend Developer US Only,https://vuejobs.com/jobs/drive-commerce-ux-and-design-focused-senior-front-end-developer?utm_source=rssfeed,Employer Drive Commerce Location United States We are building the nextgeneration ecommerce personalization platform and we are looking for you if...,2023-02-16,NaN
370,Regulatory Strategy Counsel Europe,https://cryptojobslist.com/jobs/regulatory-strategy-counsel-europe-kraken-digital-asset-exchange-amsterdam,Tags Cryptocurrency Jobs Web3 Remote Jobs Web3 Non Tech Jobs Cryptocurrency Legal Jobs Cryptocurrency Exchange Jobs Web3 Compliance Jobs Cr...,2023-02-17,Amsterdam


In [19]:
df.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_2525/3627053830.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,title,link,description,pubdate,location
count,683,683,683,443,683
unique,614,683,540,21,76
top,Senior Software Engineer,https://www.crunchboard.com/jobs/197021714-principal-software-engineer-sql-api-at-cube-dev,NaN,2023-03-08 00:00:00,NaN
freq,6,1,122,58,509
first,NaN,NaN,NaN,2023-02-16 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-08 00:00:00,NaN


In [20]:
df.dtypes

title                  object
link                   object
description            object
pubdate        datetime64[ns]
location               object
dtype: object

In [21]:
df['title'] = df['title'].astype(str)
df['link'] = df['link'].astype(str)
df['description'] = df['description'].astype(str)
df['location'] = df['location'].astype(str)

In [22]:
df.dtypes

title                  object
link                   object
description            object
pubdate        datetime64[ns]
location               object
dtype: object

##### Make a copy 

In [23]:
df1 = df.copy()

In [24]:
df['description'] = df['description'].str.slice(0, 1000)

In [25]:
# replace NaT values in the DataFrame with None
df = df.replace({np.nan: None, pd.NaT: None})

In [26]:
df

,title,link,description,pubdate,location
70,Principal software engineer SQL API at Cube Dev San Francisco CA USA,https://www.crunchboard.com/jobs/197021714-principal-software-engineer-sql-api-at-cube-dev,At Cube Dev we are building a technology stack for modern analytics If you are fascinated by the software that powers large technology companies but,2023-02-16 00:00:00,NaN
1342,Aha Senior Product Marketing Manager,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Headquarters United States URL httpwwwahaio Aha helps companies build lovable software We provide the worlds 1 product development tools Aha Ro...,2023-02-16 00:00:00,NaN
69,Senior Full Stack Developer NET at AAOS Rosemont IL USA,https://www.crunchboard.com/jobs/197057956-senior-full-stack-developer-net-at-aaos,You are an enthusiastic thought leader who coaches and mentors those around you You are highly creative and understand how to balance creativity and,2023-02-16 00:00:00,NaN
804,Voormedia is hiring for a position of Product Designer in Amsterdam Netherlands,https://dribbble.com/jobs/203058-Product-Designer?source=feed,NaN,2023-02-16 00:00:00,NaN
803,Johns Hopkins University is hiring for a position of Web Designer in Remote,https://dribbble.com/jobs/203090-Web-Designer?source=feed,NaN,2023-02-16 00:00:00,NaN
799,Riva Health is hiring for a position of Lead Product Designer Riva Health in United States Canada,https://dribbble.com/jobs/203123-Lead-Product-Designer-Riva-Health?source=feed,NaN,2023-02-16 00:00:00,NaN
883,Drupal FrontEnd Engineer Remote US amp CANADA position is open kanopistudios,https://jobs.drupal.org/kanopi-studios/job/17548,United States,2023-02-16 00:00:00,NaN
1257,Full Stack Developer,https://vuejobs.com/jobs/my-worksheet-maker-full-stack-developer?utm_source=rssfeed,Employer My Worksheet Maker Location Full Stack Developer FullTime RemotenbspAre you a skilled developer looking to use your technical expertise ...,2023-02-16 00:00:00,NaN
1256,UX and Design Focused Senior Frontend Developer US Only,https://vuejobs.com/jobs/drive-commerce-ux-and-design-focused-senior-front-end-developer?utm_source=rssfeed,Employer Drive Commerce Location United States We are building the nextgeneration ecommerce personalization platform and we are looking for you if...,2023-02-16 00:00:00,NaN
370,Regulatory Strategy Counsel Europe,https://cryptojobslist.com/jobs/regulatory-strategy-counsel-europe-kraken-digital-asset-exchange-amsterdam,Tags Cryptocurrency Jobs Web3 Remote Jobs Web3 Non Tech Jobs Cryptocurrency Legal Jobs Cryptocurrency Exchange Jobs Web3 Compliance Jobs Cr...,2023-02-17 00:00:00,Amsterdam


# PostgreSQL

### This code creates a new table per iteration

In [27]:
# create a connection to the PostgreSQL database
cnx = psycopg2.connect(user='postgres', password='3312', host='localhost', port='5432', database='postgres')

# create a cursor object
cursor = cnx.cursor()

# get the name of the next table to create
get_table_name_query = '''
    SELECT COUNT(*) FROM information_schema.tables
    WHERE table_name LIKE 'rss_%'
'''
cursor.execute(get_table_name_query)
result = cursor.fetchone()
next_table_number = result[0] + 1
next_table_name = 'rss_{}'.format(next_table_number)

# prepare the SQL query to create a new table
create_table_query = '''
    CREATE TABLE {} (
        title VARCHAR(255),
        link VARCHAR(255),
        description VARCHAR(1000),
        pubdate TIMESTAMP,
        location VARCHAR(255)
    )
'''.format(next_table_name)

# execute the create table query
cursor.execute(create_table_query)

# insert the DataFrame into the PostgreSQL database as a new table
for index, row in df.iterrows():
    insert_query = '''
        INSERT INTO {} (title, link, description, pubdate, location)
        VALUES (%s, %s, %s, %s, %s)
    '''.format(next_table_name)
    values = (row['title'], row['link'], row['description'], row['pubdate'], row['location'])
    cursor.execute(insert_query, values)

# commit the changes to the database
cnx.commit()

# close the cursor and connection
cursor.close()
cnx.close()